In [ ]:
%load_ext autoreload
%autoreload 2

import time
import os
import argparse
import cProfile
import pstats
import logging

import numpy as np
import astropy.io.fits as pyfits

from corgihowfsc.utils.howfsc_initialization import get_args_jupyter, load_files
from corgihowfsc.sensing.DefaultProbes import DefaultProbes

import eetc
from eetc.cgi_eetc import CGIEETC

import howfsc
from howfsc.control.cs import ControlStrategy
from howfsc.control.calcjtwj import JTWJMap

from howfsc.model.mode import CoronagraphMode

from howfsc.util.loadyaml import loadyaml
from howfsc.util.gitl_tools import param_order_to_list

from howfsc.gitl import howfsc_computation
from howfsc.precomp import howfsc_precomputation

from howfsc.scripts.gitlframes import sim_gitlframe

eetc_path = os.path.dirname(os.path.abspath(eetc.__file__))
howfscpath = os.path.dirname(os.path.abspath(howfsc.__file__))


defjacpath = os.path.join(os.path.dirname(howfscpath), 'jacdata')



In [ ]:
# Load arguments
args = get_args_jupyter(jacpath=defjacpath)


In [ ]:
# Initialize variables etc

otherlist = []
abs_dm1list = []
abs_dm2list = []
framelistlist = []
scalelistout = []
camlist = []

# User params
niter = args.niter
mode = args.mode
isprof = args.profile
logfile = args.logfile
fracbadpix = args.fracbadpix
nbadpacket = args.nbadpacket
nbadframe = args.nbadframe
fileout = args.fileout
stellar_vmag = args.stellarvmag
stellar_type = args.stellartype
stellar_vmag_target = args.stellarvmagtarget
stellar_type_target = args.stellartypetarget
jacpath = args.jacpath
    
modelpath, cfgfile, jacfile, cstratfile, probe0file, probe1file, probe2file, hconffile, n2clistfiles = load_files(args, howfscpath)


# cfg
cfg = CoronagraphMode(cfgfile)

In [ ]:
# Initialize default probes class
probes = DefaultProbes('default')

# Get DM lists
probefiles = {}
probefiles[0] = probe0file
probefiles[2] = probe1file
probefiles[1] = probe2file
dm1_list, dm2_list = probes.get_dm_probes(cfg, probefiles,
                      scalelist=[0.3, 0.3, 0.3, -0.3, -0.3, -0.3])

# Get probe amplitude lists (this step is slow)
plist, other = probes.get_probe_ap(cfg, dm1_list, dm2_list)